In [314]:
import openai
import time
import pandas as pd

In [315]:
client = openai.Client()

Realiza o upload de um arquivo para carregar informações ao modelo

In [316]:
file = client.files.create(
    file=open("data_upload.txt", "rb"),
    purpose="assistants"
)

Cria o assistente para acessar o arquivo

In [317]:

assistant = client.beta.assistants.create(
    name="Analista de Dados",
    instructions="Você é um analista que analisa dados sobre vendas.",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o-mini"
)


Função: Cria uma thread de conversa e envia a pergunta como mensagem.

In [318]:
#pergunta = "Qual o rating médio das vendas do supermercado"
#pergunta = "Gere um grafico de vendas por linha de produtos"
pergunta = (
    "Gere 15 perguntas para conversação avançada em ingles que não estejam contidas no arquivo data_upload. "
    "Nao repita as questões e nem mostre o numero de sequencia delas"
    "Nao gere questões similares ou parecidas com as ja existentes no arquivo data_upload.txt. Procure sempre por perguntas com temas" \
    "diferentes"
)


# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta)

C:\Users\said-\AppData\Local\Temp\ipykernel_19448\398918923.py:12: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\said-\AppData\Local\Temp\ipykernel_19448\398918923.py:13: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [319]:
#Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

C:\Users\said-\AppData\Local\Temp\ipykernel_19448\234429889.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [320]:
# Aguarda a thread rodar
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
        
    )
    print(run.status)

C:\Users\said-\AppData\Local\Temp\ipykernel_19448\2396637559.py:4: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
completed


In [321]:
# Verifica a resposta quando tivermos a resposta da thread
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens.data[0].content[0].text.value)
else:
    print(f"Erro {run.status}")

C:\Users\said-\AppData\Local\Temp\ipykernel_19448\1247652421.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


Aqui estão 15 perguntas de conversação avançada em inglês, evitando repetições das questões já contidas no arquivo:

- How do you believe social movements impact legislation and public policy in a democratic society?
- In what ways does culture shape individual behaviors and societal expectations?
- What are the long-term psychological impacts of social media on adolescents?
- How do you think the concept of beauty varies across different cultures?
- In what ways can art serve as a catalyst for social change?
- How do you perceive the relationship between mental health and physical wellness?
- What role does community play in personal development and identity formation?
- How can international cooperation address global issues such as climate change and poverty?
- What philosophical questions arise when considering the ethics of artificial intelligence?
- How can storytelling in various forms (literature, film, etc.) facilitate empathy in different cultures?
- In what ways does languag

In [322]:
# Extrai o texto da resposta/texto
texto_resposta = mensagens.data[0].content[0].text.value

# Extrai perguntas (linhas que começam com "-")
perguntas = [linha.strip("- ").strip() for linha in texto_resposta.splitlines() if linha.strip().startswith("-")]

# Grava perguntas novas no final do arquivo
if perguntas:
    with open("data_upload.txt", "a", encoding="utf-8") as f:
        for pergunta in perguntas:
            f.write(pergunta + "\n")



In [323]:
# por hora esse print ficara commitado porque ja esta sendo considerado
# mais abaixo
# print(mensagens.data[0].content[0].text.value)

In [ ]:
#analisando a logica/passos do modelo para chegar na respostas
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

C:\Users\said-\AppData\Local\Temp\ipykernel_19448\3650132559.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [325]:
for step in run_steps.data[::-1]:
    print(f"\n===Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("-" * 10)
            print(tool_call.code_interpreter.input)
            print("-" * 10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
        thread_id=thread.id,
        message_id=step.step_details.message_creation.message_id
    )

# print(message.content[0].text.value)
# comentado acim apara quando a geração do output é grafico e não texto(conforme a pergunta)


===Step tool_calls
----------
# Let's first read the contents of the uploaded file to understand the existing questions
file_path = '/mnt/data/file-HjUmWTtELo9urz7M2MSVvu'
with open(file_path, 'r', encoding='utf-8') as file:
    existing_questions = file.readlines()

# Strip newlines and create a set of existing questions for easier comparison later
existing_questions = {question.strip() for question in existing_questions}
existing_questions
----------

===Step message_creation


C:\Users\said-\AppData\Local\Temp\ipykernel_19448\1145170754.py:9: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


In [326]:

#condicional adaptado para quando a pergunta solicita texto ou grafico
#Função: Verifica se a resposta é texto ou imagem. Se for imagem, salva localmente.
if message.content[0].type == "text":
    print(message.content[0].text.value)
if message.content[0].type == "image_file":
    file_id = message.content[0].image_file.file_id
    image_data = client.files.content(file_id)
    with open(f"files/{file.id}.png", "wb") as f:
        f.write(image_data.read())
    print(f"Imagem {file_id} salva")


Aqui estão 15 perguntas de conversação avançada em inglês, evitando repetições das questões já contidas no arquivo:

- How do you believe social movements impact legislation and public policy in a democratic society?
- In what ways does culture shape individual behaviors and societal expectations?
- What are the long-term psychological impacts of social media on adolescents?
- How do you think the concept of beauty varies across different cultures?
- In what ways can art serve as a catalyst for social change?
- How do you perceive the relationship between mental health and physical wellness?
- What role does community play in personal development and identity formation?
- How can international cooperation address global issues such as climate change and poverty?
- What philosophical questions arise when considering the ethics of artificial intelligence?
- How can storytelling in various forms (literature, film, etc.) facilitate empathy in different cultures?
- In what ways does languag